## Database Setup
### Outline
- Create a database and establish a connection
- Create tables
- Load datasets
- Export datasets to MySQL tables

### Additional Considerations
- column names from governance_indicators.csv can be used directly
- OECD column names can be used
- economic freedom scores as well
- SEZ is okay

In [19]:
%pip install mysql-connector-python
%pip install mysql
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\Harb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\Harb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 1.4 MB/s eta 0:00:02
   -- ------------------------------------- 0.1/2.1 MB 1.3 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/2.1 MB 2.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.1 MB 3.8 MB/s eta 0:00:01
   ---------------- ----------------------- 0.9/2.1 MB 4.3 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 5.5 MB/s eta 0:00:01
   ------------------------------- -------- 1.7/2.1 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/298.4 kB ? eta -:--:--
   ---------------------------------------  297.0/298.4 kB 9.2 MB/s eta 0:00:01
   ------------------


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\Harb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
from sqlalchemy import create_engine

In [25]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import os

env = os.environ

MYSQL_CONFIG = {
    'host': env.get("MYSQL_HOST"),
    'user': env.get("MYSQL_USER"), 
    'password': env.get("MYSQL_PW"),
    'database': env.get("MYSQL_DB")
}

connection = mysql.connector.connect(
            host=MYSQL_CONFIG['host'],
            user=MYSQL_CONFIG['user'],
            password=MYSQL_CONFIG['password']
        )

governance_indicators = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\governance_indicators.csv")
economic_freedom_scores = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\economic_freedom_country_scores.csv")
oecd_fdi_data = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\OECD_fdi_flows_fdi_restrictiveness.csv")
global_sez = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\global_special_economic_zones.csv")
fdi_data = pd.read_csv('C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\fdi_data\\fdi_data.csv')


def create_table_from_dataframe(cursor, table_name, dataframe):
    columns = ', '.join([
        f"`{col}` VARCHAR(255)" if dtype == 'object' 
        else f"`{col}` DOUBLE" if pd.api.types.is_numeric_dtype(dataframe[col])
        else f"`{col}` TEXT" 
        for col, dtype in dataframe.dtypes.items()
    ])
    
    drop_table_sql = f"DROP TABLE IF EXISTS `{table_name}`;"
    cursor.execute(drop_table_sql)
    create_table_sql = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns});"
    cursor.execute(create_table_sql)

    placeholders = ', '.join(['%s'] * len(dataframe.columns))
    column_names = ', '.join([f"`{col}`" for col in dataframe.columns])
    insert_sql = f"INSERT INTO `{table_name}` ({column_names}) VALUES ({placeholders})"

    data_to_insert = dataframe.to_records(index=False).tolist()
    cursor.executemany(insert_sql, data_to_insert)

    print(f"Table '{table_name}' created and data inserted")


def create_database():
    try:
        
        cursor = connection.cursor()
        
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {MYSQL_CONFIG['database']}")
        cursor.execute(f"USE {MYSQL_CONFIG['database']}")

        datasets = {
            "governance_indicators": governance_indicators,
            "economic_freedom_country_scores": economic_freedom_scores,
            "global_special_economic_zones": global_sez, 
            "fdi_data": fdi_data,
            "oecd_fdi_flows_restrictiveness": oecd_fdi_data        }

        for table_name, df in datasets.items():
            create_table_from_dataframe(cursor, table_name, df)

        connection.commit()

        print("Tables done.")


    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

In [26]:
create_database()

Table 'governance_indicators' created and data inserted
Table 'economic_freedom_country_scores' created and data inserted
Table 'global_special_economic_zones' created and data inserted
Table 'fdi_data' created and data inserted
Table 'oecd_fdi_flows_restrictiveness' created and data inserted
Tables done.
